In [13]:
from torch import nn
import torch
import numpy as np

In [14]:
vocab_length = 10
vocab_dim = 3
heads = 2
embedding = nn.Embedding(vocab_length, vocab_dim)
weight_layer = nn.Linear(vocab_dim+vocab_dim, heads)
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()

In [15]:
input = torch.LongTensor([[0,1,5,6]])
labels = torch.LongTensor([2,3,1,0]) #tokens heads
n_words = input.shape[-1]
tokens = embedding(input)
tokens_pairs = torch.cat((tokens.repeat(1,n_words,1).squeeze().view(n_words*n_words, vocab_dim), 
                          tokens.repeat(1,1,n_words).squeeze().view(n_words*n_words, vocab_dim)), dim=-1).view(n_words,n_words,vocab_dim+vocab_dim).transpose(1,0)

In [16]:
attn = torch.rand(heads, n_words, n_words)
attn = attn.transpose(2,1).transpose(0,2)
tokens_h = weight_layer(tokens_pairs)
attn_tokens = attn * tokens_h
attn_sum = attn_tokens.sum(-1)
output = loss(m(attn_sum), labels)
output.backward()

#Predictions
preds = np.argmax(attn_sum.detach(), axis=1)
preds1 = np.argmax(m(attn_sum).detach(), axis=1)

In [17]:
print(preds, preds1)

tensor([0, 3, 1, 3]) tensor([0, 3, 1, 3])
